In [1]:
import torch
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

import argparse
import numpy as np
import os

from data import build_train_dataset
from gmflow.gmflow import GMFlow
from loss import flow_loss_func
from evaluate import (validate_chairs, validate_things, validate_sintel, validate_kitti,
                      create_sintel_submission, create_kitti_submission, inference_on_dir)

from utils.logger import Logger
from utils import misc
from utils.dist_utils import get_dist_info, init_dist, setup_for_distributed

In [2]:
devicde = torch.device('cuda:0')

In [3]:
model = GMFlow(feature_channels=368,
                num_scales=1,
                upsample_factor=8,
                num_head=1,
                attention_type="swin",
                ffn_dim_expansion=4,
                num_transformer_layers=3,
                ).to(devicde)

In [4]:
images_0 = torch.randn(4, 3, 256, 256).to(devicde)
images_1 = torch.randn(4, 3, 256, 256).to(devicde)

In [5]:
#feature0, feature1 = model.extract_feature(images_0, images_1)

In [6]:
#len(feature0)

In [7]:
#feature0[0].shape, feature0[1].shape

In [8]:
model.backbone.num_branch

1

In [9]:
results_dict = model(images_0, images_1,
                attn_splits_list=[2,],
                corr_radius_list=[-1,],
                prop_radius_list=[-1,]
                )

/home/ubuntu/gmflow/gmflow/position.py:39: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  dim_t = self.temperature ** (2 * (dim_t // 2) / self.num_pos_feats)
/home/ubuntu/anaconda3/envs/st_mae/lib/python3.9/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1656352660876/work/aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [10]:
len(results_dict["flow_preds"])

2

In [11]:
results_dict["flow_preds"][-1].shape

torch.Size([4, 2, 512, 512])